In [1]:
import rg
from rg.diagrams import diagram, composition_diagram, diagram_set
from rg.interaction import interaction as J 
from rg.interaction import interaction_identity
from rg.interaction import interaction_system
from rg.interaction import composite_interaction as G
from rg.theory import ftheory
from rg.graphs import composite_interaction_graph as fgraph

# Example representations
Here we show the matrix form for adding fields for arbitrary number of species (1 row per species). We display the latex symbols and also the diagrams

The diagrams are of the residuals so loops/internal structure are not shown here (see the composite examples later where we look at internal structure). 

The & operator or * operator (todo) are both used as diagram products allowing diagrams to be merged

In [2]:
j = J([[1,1], 
       [2,0]])
j

$\phi_0 \phi_1^{2} \tilde{\phi}_0$

In [3]:
j.diagram

<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="0" y="0" width="240" height="120">
      <g fill="none" stroke="black" stroke-width="1.6" stroke-linecap="round">
         <line x1="0" y1="0" x2="40" y2="0" stroke="red" stroke-dasharray="None" stroke-width:2" transform=" translate(80,50) rotate(-45.0 0, 0 )" />  <path d="M 0,0 C 2,-3.142 3,-5 5,-5 S 8,-3.142 10,0 S 13,5 15,5 S 18,3.142 20,0 C 22,-3.142 23,-5 25,-5 S 28,-3.142 30,0 S 33,5 35,5 S 38,3.142 40,0" 
                       stroke="black" stroke-dasharray="None" stroke-width:2" transform=" translate(80,50) rotate(0.0 0, 0 )" />  <path d="M 0,0 C 2,-3.142 3,-5 5,-5 S 8,-3.142 10,0 S 13,5 15,5 S 18,3.142 20,0 C 22,-3.142 23,-5 25,-5 S 28,-3.142 30,0 S 33,5 35,5 S 38,3.142 40,0" 
                       stroke="black" stroke-dasharray="None" stroke-width:2" transform=" translate(80,50) rotate(45.0 0, 0 )" />  <line x1="0" y1="0" x2="40" y2="0" stroke="red" stroke-dasharray="None" stroke-width:2" transform=" translate(80,50) rotate(-180.0 0, 0 )" /> 
      </g>
      <circle cx="80" cy="50" r="2" stroke="black" stroke-width="1" fill="black" />
    </svg>

In [4]:
transmutation = J([[1,0],[0,1]])
transmutation.diagram

<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="0" y="0" width="240" height="120">
      <g fill="none" stroke="black" stroke-width="1.6" stroke-linecap="round">
         <line x1="0" y1="0" x2="40" y2="0" stroke="red" stroke-dasharray="None" stroke-width:2" transform=" translate(80,50) rotate(0.0 0, 0 )" />  <path d="M 0,0 C 2,-3.142 3,-5 5,-5 S 8,-3.142 10,0 S 13,5 15,5 S 18,3.142 20,0 C 22,-3.142 23,-5 25,-5 S 28,-3.142 30,0 S 33,5 35,5 S 38,3.142 40,0" 
                       stroke="black" stroke-dasharray="None" stroke-width:2" transform=" translate(80,50) rotate(-180.0 0, 0 )" /> 
      </g>
      <circle cx="80" cy="50" r="2" stroke="black" stroke-width="1" fill="black" />
    </svg>

In [5]:
coagulation = J([[2,1],[0,0]])
coagulation.diagram

<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="0" y="0" width="240" height="120">
      <g fill="none" stroke="black" stroke-width="1.6" stroke-linecap="round">
         <line x1="0" y1="0" x2="40" y2="0" stroke="red" stroke-dasharray="None" stroke-width:2" transform=" translate(80,50) rotate(-30.0 0, 0 )" />  <line x1="0" y1="0" x2="40" y2="0" stroke="red" stroke-dasharray="None" stroke-width:2" transform=" translate(80,50) rotate(30.0 0, 0 )" />  <line x1="0" y1="0" x2="40" y2="0" stroke="red" stroke-dasharray="None" stroke-width:2" transform=" translate(80,50) rotate(-180.0 0, 0 )" /> 
      </g>
      <circle cx="80" cy="50" r="2" stroke="black" stroke-width="1" fill="black" />
    </svg>

In [6]:
branching = J([[1,2],[0,0]])
branching.diagram
#branching

<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="0" y="0" width="240" height="120">
      <g fill="none" stroke="black" stroke-width="1.6" stroke-linecap="round">
         <line x1="0" y1="0" x2="40" y2="0" stroke="red" stroke-dasharray="None" stroke-width:2" transform=" translate(80,50) rotate(0.0 0, 0 )" />  <line x1="0" y1="0" x2="40" y2="0" stroke="red" stroke-dasharray="None" stroke-width:2" transform=" translate(80,50) rotate(-210.0 0, 0 )" />  <line x1="0" y1="0" x2="40" y2="0" stroke="red" stroke-dasharray="None" stroke-width:2" transform=" translate(80,50) rotate(-150.0 0, 0 )" /> 
      </g>
      <circle cx="80" cy="50" r="2" stroke="black" stroke-width="1" fill="black" />
    </svg>

In [7]:
#branching.tensor

In [8]:
(branching&coagulation).diagram

symmetry factor: 2


<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="0" y="0" width="240" height="120">
      <g fill="none" stroke="black" stroke-width="1.6" stroke-linecap="round">
         <line x1="0" y1="0" x2="40" y2="0" stroke="red" stroke-dasharray="None" stroke-width:2" transform=" translate(80,50) rotate(-30.0 0, 0 )" />  <line x1="0" y1="0" x2="40" y2="0" stroke="red" stroke-dasharray="None" stroke-width:2" transform=" translate(80,50) rotate(30.0 0, 0 )" />  <line x1="0" y1="0" x2="40" y2="0" stroke="red" stroke-dasharray="None" stroke-width:2" transform=" translate(80,50) rotate(-210.0 0, 0 )" />  <line x1="0" y1="0" x2="40" y2="0" stroke="red" stroke-dasharray="None" stroke-width:2" transform=" translate(80,50) rotate(-150.0 0, 0 )" /> 
      </g>
      <circle cx="80" cy="50" r="2" stroke="black" stroke-width="1" fill="black" />
    </svg>

In [9]:
(coagulation&branching).diagram

symmetry factor: 2


<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="0" y="0" width="240" height="120">
      <g fill="none" stroke="black" stroke-width="1.6" stroke-linecap="round">
         <line x1="0" y1="0" x2="40" y2="0" stroke="red" stroke-dasharray="None" stroke-width:2" transform=" translate(80,50) rotate(0.0 0, 0 )" />  <line x1="0" y1="0" x2="40" y2="0" stroke="red" stroke-dasharray="None" stroke-width:2" transform=" translate(80,50) rotate(-180.0 0, 0 )" /> 
      </g>
      <circle cx="80" cy="50" r="2" stroke="black" stroke-width="1" fill="black" />
    </svg>

In [10]:
(branching&j).diagram

symmetry factor: 2


<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="0" y="0" width="240" height="120">
      <g fill="none" stroke="black" stroke-width="1.6" stroke-linecap="round">
         <line x1="0" y1="0" x2="40" y2="0" stroke="red" stroke-dasharray="None" stroke-width:2" transform=" translate(80,50) rotate(-45.0 0, 0 )" />  <path d="M 0,0 C 2,-3.142 3,-5 5,-5 S 8,-3.142 10,0 S 13,5 15,5 S 18,3.142 20,0 C 22,-3.142 23,-5 25,-5 S 28,-3.142 30,0 S 33,5 35,5 S 38,3.142 40,0" 
                       stroke="black" stroke-dasharray="None" stroke-width:2" transform=" translate(80,50) rotate(0.0 0, 0 )" />  <path d="M 0,0 C 2,-3.142 3,-5 5,-5 S 8,-3.142 10,0 S 13,5 15,5 S 18,3.142 20,0 C 22,-3.142 23,-5 25,-5 S 28,-3.142 30,0 S 33,5 35,5 S 38,3.142 40,0" 
                       stroke="black" stroke-dasharray="None" stroke-width:2" transform=" translate(80,50) rotate(45.0 0, 0 )" />  <line x1="0" y1="0" x2="40" y2="0" stroke="red" stroke-dasharray="None" stroke-width:2" transform=" translate(80,50) rotate(-210.0 0, 0 )" />  <line x1="0" y1="0" x2="40" y2="0" stroke="red" stroke-dasharray="None" stroke-width:2" transform=" translate(80,50) rotate(-150.0 0, 0 )" /> 
      </g>
      <circle cx="80" cy="50" r="2" stroke="black" stroke-width="1" fill="black" />
    </svg>

# Dimensional Analysis
For dimensional analysis one construct a system of terms, each term is an interaction J.

One proposes known coupling values e.g. known dimensions of propagators 

the ftheory class permutes all non fixed couplings, proposing that permutations P(n,k) are dimensionless - each permutation creates a theory

A theory fixes the dimension of fields and all couplings. The theories can then be compared and chcked via the ftheory class

In [11]:
#blank -> tilde
#BRW0
interactions = [
 J([[1,1],[0,0]]),
 J([[0,0],[1,1]]),
 J([[1,0],[0,1]]), #tau 
 J([[1,2],[0,0]]), #s
 J([[1,1],[0,1]]), #quasi-branch sigma
 J([[1,1],[1,1]]), #kappa
 J([[1,0],[1,1]]), #lambda
 ]

#BWS1 - #add two more terms chi and eta
interactions = interactions +  [
 J([[0,1],[1,2]]), #
 J([[1,0],[1,2]]), #   
]

#VRS - add two more terms annhilation
interactions = interactions +  [
 J([[1,1],[1,0]]), #
 J([[2,1],[0,1]]), #   
]

#display
[j.display() for j in interactions]

⎡                                                                             
⎣\phi₀⋅\tilde{\phi}₀, \phi₁⋅\tilde{\phi}₁, \phi₀⋅\tilde{\phi}₁, \phi₀⋅\tilde{\

     2                                                                        
phi}₀ , \phi₀⋅\tilde{\phi}₀⋅\tilde{\phi}₁, \phi₀⋅\phi₁⋅\tilde{\phi}₀⋅\tilde{\p

                                                                  2           
hi}₁, \phi₀⋅\phi₁⋅\tilde{\phi}₁, \phi₁⋅\tilde{\phi}₀⋅\tilde{\phi}₁ , \phi₀⋅\ph

                2                                  2                          
i₁⋅\tilde{\phi}₁ , \phi₀⋅\phi₁⋅\tilde{\phi}₀, \phi₀ ⋅\tilde{\phi}₀⋅\tilde{\phi

  ⎤
}₁⎦

In [12]:
Lambda = rg.T**(-1) # L**(-1*dim)
new_couplings = {J([ [0,0],  [1,1]] ): Lambda, #[field]*lambda is the inverse measure
                 J([ [1,1],  [0,0]] ): Lambda, }

#example theory
FT = ftheory(new_couplings, 
             dimensionless=[J([ [1,2],  [0,0]] ), 
                            J([ [1,1],  [0,1]] )  ])
FT.display()

⎡                                   1⎤
⎢       \phi₁⋅\tilde{\phi}₁         ─⎥
⎢                                   T⎥
⎢                                    ⎥
⎢                                   1⎥
⎢       \phi₀⋅\tilde{\phi}₀         ─⎥
⎢                                   T⎥
⎢                                    ⎥
⎢                         2          ⎥
⎢      \phi₀⋅\tilde{\phi}₀          1⎥
⎢                                    ⎥
⎣\phi₀⋅\tilde{\phi}₀⋅\tilde{\phi}₁  1⎦

In [13]:
FT.interpret_dimensions(interactions)

⎡                                            -d  ⎤
⎢          \phi₀⋅\tilde{\phi}₀              L    ⎥
⎢                                                ⎥
⎢                                            -d  ⎥
⎢          \phi₁⋅\tilde{\phi}₁              L    ⎥
⎢                                                ⎥
⎢                                            -d  ⎥
⎢          \phi₀⋅\tilde{\phi}₁              L    ⎥
⎢                                                ⎥
⎢                                            -d  ⎥
⎢                            2              L    ⎥
⎢         \phi₀⋅\tilde{\phi}₀               ───  ⎥
⎢                                            T   ⎥
⎢                                                ⎥
⎢                                            -d  ⎥
⎢                                           L    ⎥
⎢   \phi₀⋅\tilde{\phi}₀⋅\tilde{\phi}₁       ───  ⎥
⎢                                            T   ⎥
⎢                                                ⎥
⎢                              

In [14]:
FT.interpret_couplings(interactions, l_power_dim=4)

⎡                                         1 ⎤
⎢          \phi₀⋅\tilde{\phi}₀            ──⎥
⎢                                          2⎥
⎢                                         L ⎥
⎢                                           ⎥
⎢                                         1 ⎥
⎢          \phi₁⋅\tilde{\phi}₁            ──⎥
⎢                                          2⎥
⎢                                         L ⎥
⎢                                           ⎥
⎢                                         1 ⎥
⎢          \phi₀⋅\tilde{\phi}₁            ──⎥
⎢                                          2⎥
⎢                                         L ⎥
⎢                                           ⎥
⎢                            2              ⎥
⎢         \phi₀⋅\tilde{\phi}₀             1 ⎥
⎢                                           ⎥
⎢   \phi₀⋅\tilde{\phi}₀⋅\tilde{\phi}₁     1 ⎥
⎢                                           ⎥
⎢                                          2⎥
⎢\phi₀⋅\phi₁⋅\tilde{\phi}₀⋅\tilde{

In [15]:
#we can create a bunch of theories by permuting dimensionless couplings given the input known field dimensions and all terms
cs  = ftheory.theories(interactions, new_couplings)

In [16]:
import pandas as pd
dfs = []
for k in cs:
    M = k.interpret_couplings(interactions)
    df = pd.DataFrame(M.tolist(), columns=["vertex", "coupling"]).set_index("vertex")
    dfs.append(df)
dfs = pd.concat(dfs,axis=1).reset_index()
from sympy import Matrix
Matrix(dfs.as_matrix())
#dfs[0]

⎡                                         1   1    1     1     1    1   1     
⎢          \phi₀⋅\tilde{\phi}₀            ─   ─    ─     ─     ─    ─   ─     
⎢                                         T   T    T     T     T    T   T     
⎢                                                                             
⎢                                         1   1    1     1     1    1   1     
⎢          \phi₁⋅\tilde{\phi}₁            ─   ─    ─     ─     ─    ─   ─     
⎢                                         T   T    T     T     T    T   T     
⎢                                                                             
⎢                                                                             
⎢                                                                   1   1     
⎢          \phi₀⋅\tilde{\phi}₁            1   1    1     1     1    ─   ──   L
⎢                                                                   T    2    
⎢                                                   

In [17]:
ftheory.matrices(cs).dimensionless_terms().T

⎡                                                                             
⎢\phi₀⋅\tilde{\phi}₁          \phi₀⋅\tilde{\phi}₁            \phi₀⋅\tilde{\phi
⎢                                                                             
⎢                   2                                                         
⎣\phi₀⋅\tilde{\phi}₀   \phi₀⋅\tilde{\phi}₀⋅\tilde{\phi}₁  \phi₀⋅\phi₁⋅\tilde{\

                                                                              
}₁            \phi₀⋅\tilde{\phi}₁             \phi₀⋅\tilde{\phi}₁           \p
                                                                              
                                        2                                     
phi}₁  \phi₁⋅\tilde{\phi}₀⋅\tilde{\phi}₁   \phi₀⋅\phi₁⋅\tilde{\phi}₀  \phi₀⋅\t

                 2                                   2                        
hi₀⋅\tilde{\phi}₀                 \phi₀⋅\tilde{\phi}₀             \phi₀⋅\tilde
                                                  

In [18]:
#we can display a matrix for all theories for the values of the fields \phi_0, \tilde{\phi}_0, \phi_1, \tilde{\phi_1}
ftheory.matrices(cs).general_form
#below terms are in the form ud+v
#propagators always compensate each other in d=0 - that means i think they are rate-like
#you never have something that is just a positive v without some u? 
#you sometimes but rarely have any positive u
#For u,v; 0,0 always goes with -1,0
#you only have powers of 2 for u,v
#d dependence for creation or annihilation fields mostly agree between species - 
#  - if i was think of this as some sort of polarity - i want the two species to maybe agree
#  - that way propagation between species (transmutation) is as 'well behaved' as the bare propagators (so to speak)
#  - specifically i mean if u > 0 or v > 0 for one species, must be the same for the other

#todo - what do all the degenerate theories have in common (including/not including the non integer and negative dimensions)
#i notice two of the theories are exactly the same below too e.g. 1 and 3 which is weird but this explains the negatives 
#  - maybe it is the same thing actually
#from the measure, we know that we will have u,v = 1,2 (or negateive on the RHS).
#  -  Therefore, we know that we need at least +u and v cancelling the 2 - this creates our baseline

⎡ -d + 2      -2      -d + 4     -4  ⎤
⎢                                    ⎥
⎢   -d        0       -d + 2     -2  ⎥
⎢                                    ⎥
⎢   -2      -d + 2      0        -d  ⎥
⎢                                    ⎥
⎢   -d        0       -d + 2     -2  ⎥
⎢                                    ⎥
⎢   -4      -d + 4      -2     -d + 2⎥
⎢                                    ⎥
⎢ -d + 2      -2      -d + 2     -2  ⎥
⎢                                    ⎥
⎢ -d + 2      -2        -d       0   ⎥
⎢                                    ⎥
⎢ -d + 2      -2     -2⋅d + 4  d - 4 ⎥
⎢                                    ⎥
⎢ -d + 2      -2        -2     -d + 2⎥
⎢                                    ⎥
⎢ -d + 2      -2     -2⋅d + 4  d - 4 ⎥
⎢                                    ⎥
⎢   -2      -d + 2    -d + 2     -2  ⎥
⎢                                    ⎥
⎢   -d        0       -d + 2     -2  ⎥
⎢                                    ⎥
⎢   0         -d      -d + 2     -2  ⎥
⎢                        

In [19]:
# from itertools import product

# M = [0,1,2,3]
# def multiradix_product(M):
#     return product(*(range(x) for x in M))

# list(multiradix_product(M))
# #for a in multiradix_recursive(M):print(a)

In [20]:
#and we can evalaute the values at a certain d, last column is the inverse measure L^dT^1 with T=L^2 at d=d_c
ftheory.matrices(cs).criterion(4)

⎡2  2  0  4  -6⎤
⎢              ⎥
⎢4  0  2  2  -6⎥
⎢              ⎥
⎢2  2  0  4  -6⎥
⎢              ⎥
⎢4  0  2  2  -6⎥
⎢              ⎥
⎢4  0  2  2  -6⎥
⎢              ⎥
⎢2  2  2  2  -6⎥
⎢              ⎥
⎢2  2  4  0  -6⎥
⎢              ⎥
⎢2  2  4  0  -6⎥
⎢              ⎥
⎢2  2  2  2  -6⎥
⎢              ⎥
⎢2  2  4  0  -6⎥
⎢              ⎥
⎢2  2  2  2  -6⎥
⎢              ⎥
⎢4  0  2  2  -6⎥
⎢              ⎥
⎢0  4  2  2  -6⎥
⎢              ⎥
⎢0  4  2  2  -6⎥
⎢              ⎥
⎢2  2  4  0  -6⎥
⎢              ⎥
⎢2  2  4  0  -6⎥
⎢              ⎥
⎢2  2  2  2  -6⎥
⎢              ⎥
⎢2  2  4  0  -6⎥
⎢              ⎥
⎢4  4  6  2  -6⎥
⎢              ⎥
⎢4  0  2  2  -6⎥
⎢              ⎥
⎢4  4  6  2  -6⎥
⎢              ⎥
⎢0  4  2  2  -6⎥
⎢              ⎥
⎣0  4  2  2  -6⎦

# Composing and displaying diagrams
Work in progress. Lots of different ways to render things - starting with the basics, the ones that make development clearer.

In [21]:
G(branching)*G(coagulation)

<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="0" y="0" width="500" height="120">
      <g fill="none" stroke="black" stroke-width="1.6" stroke-linecap="round"> <circle cx="70" cy="50" r="3" stroke="black" stroke-width="1" fill="black" /> <circle cx="20" cy="50" r="3" stroke="black" stroke-width="1" fill="black" /> <circle cx="52" cy="41" r="2" stroke="green" stroke-width="1" fill='green' /> <circle cx="52" cy="59" r="2" stroke="green" stroke-width="1" fill='green' /> <circle cx="88" cy="50" r="2" stroke="green" stroke-width="1"  /> <circle cx="38" cy="41" r="2" stroke="green" stroke-width="1"  /> <circle cx="38" cy="59" r="2" stroke="green" stroke-width="1"  /> <circle cx="2" cy="50" r="2" stroke="green" stroke-width="1" fill='green' /> <line x1="52" y1="41" x2="70" y2="50" stroke="green" stroke-width:2"  stroke-dasharray='2,2' /><line x1="52" y1="41" x2="38" y2="41" stroke="green" stroke-width:2"  stroke-dasharray='2,2' /><line x1="38" y1="41" x2="20" y2="50" stroke="green" stroke-width:2"  stroke-dasharray='2,2' /><line x1="52" y1="59" x2="70" y2="50" stroke="green" stroke-width:2"  stroke-dasharray='2,2' /><line x1="52" y1="59" x2="38" y2="59" stroke="green" stroke-width:2"  stroke-dasharray='2,2' /><line x1="38" y1="59" x2="20" y2="50" stroke="green" stroke-width:2"  stroke-dasharray='2,2' /><line x1="88" y1="50" x2="70" y2="50" stroke="green" stroke-width:2"   /><line x1="2" y1="50" x2="20" y2="50" stroke="green" stroke-width:2"   />  </g> </svg>

In [22]:
T = G(branching)*G(transmutation)*G(j)
T

<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="0" y="0" width="500" height="120">
      <g fill="none" stroke="black" stroke-width="1.6" stroke-linecap="round"> <circle cx="120" cy="50" r="3" stroke="black" stroke-width="1" fill="black" /> <circle cx="70" cy="50" r="3" stroke="black" stroke-width="1" fill="black" /> <circle cx="20" cy="50" r="3" stroke="black" stroke-width="1" fill="black" /> <circle cx="102" cy="41" r="2" stroke="green" stroke-width="1" fill='green' /> <circle cx="102" cy="59" r="2" stroke="green" stroke-width="1" fill='green' /> <circle cx="138" cy="50" r="2" stroke="green" stroke-width="1"  /> <circle cx="88" cy="50" r="2" stroke="green" stroke-width="1"  /> <circle cx="52" cy="50" r="2" stroke="orange" stroke-width="1" fill='orange' /> <circle cx="38" cy="32" r="2" stroke="green" stroke-width="1"  /> <circle cx="38" cy="50" r="2" stroke="orange" stroke-width="1"  /> <circle cx="2" cy="50" r="2" stroke="green" stroke-width="1" fill='green' /> <circle cx="38" cy="68" r="2" stroke="orange" stroke-width="1"  /> <line x1="102" y1="41" x2="120" y2="50" stroke="green" stroke-width:2"  stroke-dasharray='2,2' /><line x1="102" y1="41" x2="88" y2="50" stroke="green" stroke-width:2"  stroke-dasharray='2,2' /><line x1="88" y1="50" x2="70" y2="50" stroke="green" stroke-width:2"  stroke-dasharray='2,2' /><line x1="102" y1="59" x2="120" y2="50" stroke="green" stroke-width:2"  stroke-dasharray='2,2' /><line x1="102" y1="59" x2="38" y2="32" stroke="green" stroke-width:2"  stroke-dasharray='2,2' /><line x1="38" y1="32" x2="20" y2="50" stroke="green" stroke-width:2"  stroke-dasharray='2,2' /><line x1="52" y1="50" x2="70" y2="50" stroke="orange" stroke-width:2"  stroke-dasharray='2,2' /><line x1="52" y1="50" x2="38" y2="50" stroke="orange" stroke-width:2"  stroke-dasharray='2,2' /><line x1="38" y1="50" x2="20" y2="50" stroke="orange" stroke-width:2"  stroke-dasharray='2,2' /><line x1="138" y1="50" x2="120" y2="50" stroke="green" stroke-width:2"   /><line x1="2" y1="50" x2="20" y2="50" stroke="green" stroke-width:2"   /><line x1="38" y1="68" x2="20" y2="50" stroke="orange" stroke-width:2"   />  </g> </svg>

In [23]:
T.edges

[{'edge': [0, 1], 'edge_species': 0},
 {'edge': [0, 2], 'edge_species': 0},
 {'edge': [2, 1], 'edge_species': 1}]

In [52]:
#def vertex_residual_contribution(self):[t.sum() for t in self.tensor]
T = G(branching)*G(transmutation)
T =  G(branching)*G(transmutation)*G(j)
FG = fgraph(T,True)#.betti_number()
FG

$\left[\begin{smallmatrix}-1 & -1 & 0 & -1 & 0\\1 & 0 & 1 & 0 & 0\\0 & 1 & -1 & 0 & -1\\0 & 0 & 0 & 1 & 1\end{smallmatrix}\right]$

In [53]:
FG.graph_polynomials()

(x₁⋅x₄⋅x₅ + x₂⋅x₄⋅x₅ + x₃⋅x₄⋅x₅, x₁⋅x₂⋅x₄⋅x₅ + x₂⋅x₃⋅x₄⋅x₅)

In [26]:
T0 = G(branching)*G(transmutation)
T0

<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="0" y="0" width="500" height="120">
      <g fill="none" stroke="black" stroke-width="1.6" stroke-linecap="round"> <circle cx="70" cy="50" r="3" stroke="black" stroke-width="1" fill="black" /> <circle cx="20" cy="50" r="3" stroke="black" stroke-width="1" fill="black" /> <circle cx="52" cy="41" r="2" stroke="green" stroke-width="1" fill='green' /> <circle cx="52" cy="59" r="2" stroke="green" stroke-width="1" fill='green' /> <circle cx="88" cy="50" r="2" stroke="green" stroke-width="1"  /> <circle cx="38" cy="50" r="2" stroke="green" stroke-width="1"  /> <circle cx="2" cy="50" r="2" stroke="orange" stroke-width="1" fill='orange' /> <line x1="52" y1="41" x2="70" y2="50" stroke="green" stroke-width:2"  stroke-dasharray='2,2' /><line x1="52" y1="41" x2="38" y2="50" stroke="green" stroke-width:2"  stroke-dasharray='2,2' /><line x1="38" y1="50" x2="20" y2="50" stroke="green" stroke-width:2"  stroke-dasharray='2,2' /><line x1="52" y1="59" x2="70" y2="50" stroke="green" stroke-width:2"   /><line x1="88" y1="50" x2="70" y2="50" stroke="green" stroke-width:2"   /><line x1="2" y1="50" x2="20" y2="50" stroke="orange" stroke-width:2"   />  </g> </svg>

In [27]:
collection = [G(branching)*G(transmutation),
             G(branching)*G(coagulation),
             G(branching)*G(transmutation)*G(j)]
diagram_set(collection)

<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="0" y="0" width="500" height="210">['<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="0" y="0" width="500" height="120">\n      <g fill="none" stroke="black" stroke-width="1.6" stroke-linecap="round"> <circle cx="70" cy="50" r="3" stroke="black" stroke-width="1" fill="black" /> <circle cx="20" cy="50" r="3" stroke="black" stroke-width="1" fill="black" /> <circle cx="52" cy="41" r="2" stroke="green" stroke-width="1" fill=\'green\' /> <circle cx="52" cy="59" r="2" stroke="green" stroke-width="1" fill=\'green\' /> <circle cx="88" cy="50" r="2" stroke="green" stroke-width="1"  /> <circle cx="38" cy="50" r="2" stroke="green" stroke-width="1"  /> <circle cx="2" cy="50" r="2" stroke="orange" stroke-width="1" fill=\'orange\' /> <line x1="52" y1="41" x2="70" y2="50" stroke="green" stroke-width:2"  stroke-dasharray=\'2,2\' /><line x1="52" y1="41" x2="38" y2="50" stroke="green" stroke-width:2"  stroke-dasharray=\'2,2\' /><line x1="38" y1="50" x2="20" y2="50" stroke="green" stroke-width:2"  stroke-dasharray=\'2,2\' /><line x1="52" y1="59" x2="70" y2="50" stroke="green" stroke-width:2"   /><line x1="88" y1="50" x2="70" y2="50" stroke="green" stroke-width:2"   /><line x1="2" y1="50" x2="20" y2="50" stroke="orange" stroke-width:2"   />  </g> </svg>', '<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="0" y="50" width="500" height="120">\n      <g fill="none" stroke="black" stroke-width="1.6" stroke-linecap="round"> <circle cx="70" cy="50" r="3" stroke="black" stroke-width="1" fill="black" /> <circle cx="20" cy="50" r="3" stroke="black" stroke-width="1" fill="black" /> <circle cx="52" cy="41" r="2" stroke="green" stroke-width="1" fill=\'green\' /> <circle cx="52" cy="59" r="2" stroke="green" stroke-width="1" fill=\'green\' /> <circle cx="88" cy="50" r="2" stroke="green" stroke-width="1"  /> <circle cx="38" cy="41" r="2" stroke="green" stroke-width="1"  /> <circle cx="38" cy="59" r="2" stroke="green" stroke-width="1"  /> <circle cx="2" cy="50" r="2" stroke="green" stroke-width="1" fill=\'green\' /> <line x1="52" y1="41" x2="70" y2="50" stroke="green" stroke-width:2"  stroke-dasharray=\'2,2\' /><line x1="52" y1="41" x2="38" y2="41" stroke="green" stroke-width:2"  stroke-dasharray=\'2,2\' /><line x1="38" y1="41" x2="20" y2="50" stroke="green" stroke-width:2"  stroke-dasharray=\'2,2\' /><line x1="52" y1="59" x2="70" y2="50" stroke="green" stroke-width:2"  stroke-dasharray=\'2,2\' /><line x1="52" y1="59" x2="38" y2="59" stroke="green" stroke-width:2"  stroke-dasharray=\'2,2\' /><line x1="38" y1="59" x2="20" y2="50" stroke="green" stroke-width:2"  stroke-dasharray=\'2,2\' /><line x1="88" y1="50" x2="70" y2="50" stroke="green" stroke-width:2"   /><line x1="2" y1="50" x2="20" y2="50" stroke="green" stroke-width:2"   />  </g> </svg>', '<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="0" y="100" width="500" height="120">\n      <g fill="none" stroke="black" stroke-width="1.6" stroke-linecap="round"> <circle cx="120" cy="50" r="3" stroke="black" stroke-width="1" fill="black" /> <circle cx="70" cy="50" r="3" stroke="black" stroke-width="1" fill="black" /> <circle cx="20" cy="50" r="3" stroke="black" stroke-width="1" fill="black" /> <circle cx="102" cy="41" r="2" stroke="green" stroke-width="1" fill=\'green\' /> <circle cx="102" cy="59" r="2" stroke="green" stroke-width="1" fill=\'green\' /> <circle cx="138" cy="50" r="2" stroke="green" stroke-width="1"  /> <circle cx="88" cy="50" r="2" stroke="green" stroke-width="1"  /> <circle cx="52" cy="50" r="2" stroke="orange" stroke-width="1" fill=\'orange\' /> <circle cx="38" cy="32" r="2" stroke="green" stroke-width="1"  /> <circle cx="38" cy="50" r="2" stroke="orange" stroke-width="1"  /> <circle cx="2" cy="50" r="2" stroke="green" stroke-width="1" fill=\'green\' />

In [28]:
collection = [G(branching)*G(transmutation),
             G(branching)*G(coagulation),
             G(branching)*G(transmutation)*G(j)]
collection = [c for c in collection if fgraph(c).is_1PI]
diagram_set(collection)

<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="0" y="0" width="500" height="160">['<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="0" y="0" width="500" height="120">\n      <g fill="none" stroke="black" stroke-width="1.6" stroke-linecap="round"> <circle cx="70" cy="50" r="3" stroke="black" stroke-width="1" fill="black" /> <circle cx="20" cy="50" r="3" stroke="black" stroke-width="1" fill="black" /> <circle cx="52" cy="41" r="2" stroke="green" stroke-width="1" fill=\'green\' /> <circle cx="52" cy="59" r="2" stroke="green" stroke-width="1" fill=\'green\' /> <circle cx="88" cy="50" r="2" stroke="green" stroke-width="1"  /> <circle cx="38" cy="41" r="2" stroke="green" stroke-width="1"  /> <circle cx="38" cy="59" r="2" stroke="green" stroke-width="1"  /> <circle cx="2" cy="50" r="2" stroke="green" stroke-width="1" fill=\'green\' /> <line x1="52" y1="41" x2="70" y2="50" stroke="green" stroke-width:2"  stroke-dasharray=\'2,2\' /><line x1="52" y1="41" x2="38" y2="41" stroke="green" stroke-width:2"  stroke-dasharray=\'2,2\' /><line x1="38" y1="41" x2="20" y2="50" stroke="green" stroke-width:2"  stroke-dasharray=\'2,2\' /><line x1="52" y1="59" x2="70" y2="50" stroke="green" stroke-width:2"  stroke-dasharray=\'2,2\' /><line x1="52" y1="59" x2="38" y2="59" stroke="green" stroke-width:2"  stroke-dasharray=\'2,2\' /><line x1="38" y1="59" x2="20" y2="50" stroke="green" stroke-width:2"  stroke-dasharray=\'2,2\' /><line x1="88" y1="50" x2="70" y2="50" stroke="green" stroke-width:2"   /><line x1="2" y1="50" x2="20" y2="50" stroke="green" stroke-width:2"   />  </g> </svg>', '<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="0" y="50" width="500" height="120">\n      <g fill="none" stroke="black" stroke-width="1.6" stroke-linecap="round"> <circle cx="120" cy="50" r="3" stroke="black" stroke-width="1" fill="black" /> <circle cx="70" cy="50" r="3" stroke="black" stroke-width="1" fill="black" /> <circle cx="20" cy="50" r="3" stroke="black" stroke-width="1" fill="black" /> <circle cx="102" cy="41" r="2" stroke="green" stroke-width="1" fill=\'green\' /> <circle cx="102" cy="59" r="2" stroke="green" stroke-width="1" fill=\'green\' /> <circle cx="138" cy="50" r="2" stroke="green" stroke-width="1"  /> <circle cx="88" cy="50" r="2" stroke="green" stroke-width="1"  /> <circle cx="52" cy="50" r="2" stroke="orange" stroke-width="1" fill=\'orange\' /> <circle cx="38" cy="32" r="2" stroke="green" stroke-width="1"  /> <circle cx="38" cy="50" r="2" stroke="orange" stroke-width="1"  /> <circle cx="2" cy="50" r="2" stroke="green" stroke-width="1" fill=\'green\' /> <circle cx="38" cy="68" r="2" stroke="orange" stroke-width="1"  /> <line x1="102" y1="41" x2="120" y2="50" stroke="green" stroke-width:2"  stroke-dasharray=\'2,2\' /><line x1="102" y1="41" x2="88" y2="50" stroke="green" stroke-width:2"  stroke-dasharray=\'2,2\' /><line x1="88" y1="50" x2="70" y2="50" stroke="green" stroke-width:2"  stroke-dasharray=\'2,2\' /><line x1="102" y1="59" x2="120" y2="50" stroke="green" stroke-width:2"  stroke-dasharray=\'2,2\' /><line x1="102" y1="59" x2="38" y2="32" stroke="green" stroke-width:2"  stroke-dasharray=\'2,2\' /><line x1="38" y1="32" x2="20" y2="50" stroke="green" stroke-width:2"  stroke-dasharray=\'2,2\' /><line x1="52" y1="50" x2="70" y2="50" stroke="orange" stroke-width:2"  stroke-dasharray=\'2,2\' /><line x1="52" y1="50" x2="38" y2="50" stroke="orange" stroke-width:2"  stroke-dasharray=\'2,2\' /><line x1="38" y1="50" x2="20" y2="50" stroke="orange" stroke-width:2"  stroke-dasharray=\'2,2\' /><line x1="138" y1="50" x2="120" y2="50" stroke="green" stroke-width:2"   /><line x1="2" y1="50" x2="20" y2="50" stroke="green" stroke-width:2"   /><line x1="38" y1="68" x2="20" y2="50" stroke="orange" stroke-width:2"   />  </g> </svg>']</svg>

In [29]:
collection[-1].residual_interaction.diagram

<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="0" y="0" width="240" height="120">
      <g fill="none" stroke="black" stroke-width="1.6" stroke-linecap="round">
         <line x1="0" y1="0" x2="40" y2="0" stroke="red" stroke-dasharray="None" stroke-width:2" transform=" translate(80,50) rotate(-30.0 0, 0 )" />  <path d="M 0,0 C 2,-3.142 3,-5 5,-5 S 8,-3.142 10,0 S 13,5 15,5 S 18,3.142 20,0 C 22,-3.142 23,-5 25,-5 S 28,-3.142 30,0 S 33,5 35,5 S 38,3.142 40,0" 
                       stroke="black" stroke-dasharray="None" stroke-width:2" transform=" translate(80,50) rotate(30.0 0, 0 )" />  <line x1="0" y1="0" x2="40" y2="0" stroke="red" stroke-dasharray="None" stroke-width:2" transform=" translate(80,50) rotate(-180.0 0, 0 )" /> 
      </g>
      <circle cx="80" cy="50" r="2" stroke="black" stroke-width="1" fill="black" />
    </svg>

## Permutations

In [30]:
#interactions[2:]

In [31]:
import itertools,functools

def _combine(set_of_interactions):
        def _merge(a,b): 
            if a is None:return b
            if b is None:return a
            return a * b
        return functools.reduce(_merge, set_of_interactions, None)
    
def graph_permutations(primitives, loop_orders = [0,1], max_k=3):
    if not isinstance(loop_orders, list):loop_orders = [loop_orders]
    l = [] 
    
    for i in range(2,max_k+1):
        for tup in list(itertools.permutations(primitives,i)):
            res = _combine(list(tup))
            if res.loops in loop_orders:
                l.append(res)
    #todo : define uniqueness and validity
    return list(set(l))

#simple sample system
res = graph_permutations([G(branching), G(transmutation),G(coagulation),G(j)])
#our vicious walker system
res = graph_permutations([G(j) for j in interactions[2:]])

len(res)

276

In [32]:
diagram_set([G(j) for j in interactions[2:]])

<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="0" y="0" width="500" height="510">['<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="0" y="0" width="500" height="120">\n      <g fill="none" stroke="black" stroke-width="1.6" stroke-linecap="round"> <circle cx="20" cy="50" r="3" stroke="black" stroke-width="1" fill="black" /> <circle cx="38" cy="50" r="2" stroke="green" stroke-width="1"  /> <circle cx="2" cy="50" r="2" stroke="orange" stroke-width="1" fill=\'orange\' /> <line x1="38" y1="50" x2="20" y2="50" stroke="green" stroke-width:2"   /><line x1="2" y1="50" x2="20" y2="50" stroke="orange" stroke-width:2"   />  </g> </svg>', '<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="0" y="50" width="500" height="120">\n      <g fill="none" stroke="black" stroke-width="1.6" stroke-linecap="round"> <circle cx="20" cy="50" r="3" stroke="black" stroke-width="1" fill="black" /> <circle cx="2" cy="41" r="2" stroke="green" stroke-width="1" fill=\'green\' /> <circle cx="2" cy="59" r="2" stroke="green" stroke-width="1" fill=\'green\' /> <circle cx="38" cy="50" r="2" stroke="green" stroke-width="1"  /> <line x1="2" y1="41" x2="20" y2="50" stroke="green" stroke-width:2"   /><line x1="2" y1="59" x2="20" y2="50" stroke="green" stroke-width:2"   /><line x1="38" y1="50" x2="20" y2="50" stroke="green" stroke-width:2"   />  </g> </svg>', '<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="0" y="100" width="500" height="120">\n      <g fill="none" stroke="black" stroke-width="1.6" stroke-linecap="round"> <circle cx="20" cy="50" r="3" stroke="black" stroke-width="1" fill="black" /> <circle cx="2" cy="41" r="2" stroke="green" stroke-width="1" fill=\'green\' /> <circle cx="38" cy="50" r="2" stroke="green" stroke-width="1"  /> <circle cx="2" cy="59" r="2" stroke="orange" stroke-width="1" fill=\'orange\' /> <line x1="2" y1="41" x2="20" y2="50" stroke="green" stroke-width:2"   /><line x1="38" y1="50" x2="20" y2="50" stroke="green" stroke-width:2"   /><line x1="2" y1="59" x2="20" y2="50" stroke="orange" stroke-width:2"   />  </g> </svg>', '<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="0" y="150" width="500" height="120">\n      <g fill="none" stroke="black" stroke-width="1.6" stroke-linecap="round"> <circle cx="20" cy="50" r="3" stroke="black" stroke-width="1" fill="black" /> <circle cx="2" cy="41" r="2" stroke="green" stroke-width="1" fill=\'green\' /> <circle cx="38" cy="41" r="2" stroke="green" stroke-width="1"  /> <circle cx="2" cy="59" r="2" stroke="orange" stroke-width="1" fill=\'orange\' /> <circle cx="38" cy="59" r="2" stroke="orange" stroke-width="1"  /> <line x1="2" y1="41" x2="20" y2="50" stroke="green" stroke-width:2"   /><line x1="38" y1="41" x2="20" y2="50" stroke="green" stroke-width:2"   /><line x1="2" y1="59" x2="20" y2="50" stroke="orange" stroke-width:2"   /><line x1="38" y1="59" x2="20" y2="50" stroke="orange" stroke-width:2"   />  </g> </svg>', '<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="0" y="200" width="500" height="120">\n      <g fill="none" stroke="black" stroke-width="1.6" stroke-linecap="round"> <circle cx="20" cy="50" r="3" stroke="black" stroke-width="1" fill="black" /> <circle cx="38" cy="41" r="2" stroke="green" stroke-width="1"  /> <circle cx="2" cy="50" r="2" stroke="orange" stroke-width="1" fill=\'orange\' /> <circle cx="38" cy="59" r="2" stroke="orange" stroke-width="1"  /> <line x1="38" y1="41" x2="20" y2="50" stroke="green" stroke-width:2"   /><line x1="2" y1="50" x2="20" y2="50" stroke="orange" stroke-width:2"   /><line x1="38" y1="59" x2="20" y2="50" stroke="orange" stroke-width:2"   />  </g> </svg>', '<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="0" y="250"

In [33]:
#iteratively find all tree-level new residuals
#this is a bit messy at the moment, I am still deciding how to organise this stuff
l = 0
res_dict = {}
basis = [G(j) for j in interactions[2:]]
for b in basis:
    res_dict[b.residual_interaction] = True
print("basis init length", len(basis))
temp = graph_permutations(basis, loop_orders=[0], max_k=2)
#this would be a recursive step
for b in temp:
    resid = b.residual_interaction
    if resid.tensor.sum() <= 3:
        res_dict[resid] = True
basis = [G(j) for j in res_dict.keys()]
len(basis)

basis init length 9


12

In [34]:
temp = graph_permutations(basis, loop_orders=[0], max_k=2)
#this would be a recursive step
for b in temp:
    resid = b.residual_interaction
    if resid.tensor.sum() <= 3:
        res_dict[resid] = True
basis = [G(j) for j in res_dict.keys()]
len(basis)

12

In [35]:
diagram_set(basis)

<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="0" y="0" width="500" height="660">['<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="0" y="0" width="500" height="120">\n      <g fill="none" stroke="black" stroke-width="1.6" stroke-linecap="round"> <circle cx="20" cy="50" r="3" stroke="black" stroke-width="1" fill="black" /> <circle cx="38" cy="50" r="2" stroke="green" stroke-width="1"  /> <circle cx="2" cy="50" r="2" stroke="orange" stroke-width="1" fill=\'orange\' /> <line x1="38" y1="50" x2="20" y2="50" stroke="green" stroke-width:2"   /><line x1="2" y1="50" x2="20" y2="50" stroke="orange" stroke-width:2"   />  </g> </svg>', '<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="0" y="50" width="500" height="120">\n      <g fill="none" stroke="black" stroke-width="1.6" stroke-linecap="round"> <circle cx="20" cy="50" r="3" stroke="black" stroke-width="1" fill="black" /> <circle cx="2" cy="41" r="2" stroke="green" stroke-width="1" fill=\'green\' /> <circle cx="2" cy="59" r="2" stroke="green" stroke-width="1" fill=\'green\' /> <circle cx="38" cy="50" r="2" stroke="green" stroke-width="1"  /> <line x1="2" y1="41" x2="20" y2="50" stroke="green" stroke-width:2"   /><line x1="2" y1="59" x2="20" y2="50" stroke="green" stroke-width:2"   /><line x1="38" y1="50" x2="20" y2="50" stroke="green" stroke-width:2"   />  </g> </svg>', '<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="0" y="100" width="500" height="120">\n      <g fill="none" stroke="black" stroke-width="1.6" stroke-linecap="round"> <circle cx="20" cy="50" r="3" stroke="black" stroke-width="1" fill="black" /> <circle cx="2" cy="41" r="2" stroke="green" stroke-width="1" fill=\'green\' /> <circle cx="38" cy="50" r="2" stroke="green" stroke-width="1"  /> <circle cx="2" cy="59" r="2" stroke="orange" stroke-width="1" fill=\'orange\' /> <line x1="2" y1="41" x2="20" y2="50" stroke="green" stroke-width:2"   /><line x1="38" y1="50" x2="20" y2="50" stroke="green" stroke-width:2"   /><line x1="2" y1="59" x2="20" y2="50" stroke="orange" stroke-width:2"   />  </g> </svg>', '<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="0" y="150" width="500" height="120">\n      <g fill="none" stroke="black" stroke-width="1.6" stroke-linecap="round"> <circle cx="20" cy="50" r="3" stroke="black" stroke-width="1" fill="black" /> <circle cx="2" cy="41" r="2" stroke="green" stroke-width="1" fill=\'green\' /> <circle cx="38" cy="41" r="2" stroke="green" stroke-width="1"  /> <circle cx="2" cy="59" r="2" stroke="orange" stroke-width="1" fill=\'orange\' /> <circle cx="38" cy="59" r="2" stroke="orange" stroke-width="1"  /> <line x1="2" y1="41" x2="20" y2="50" stroke="green" stroke-width:2"   /><line x1="38" y1="41" x2="20" y2="50" stroke="green" stroke-width:2"   /><line x1="2" y1="59" x2="20" y2="50" stroke="orange" stroke-width:2"   /><line x1="38" y1="59" x2="20" y2="50" stroke="orange" stroke-width:2"   />  </g> </svg>', '<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="0" y="200" width="500" height="120">\n      <g fill="none" stroke="black" stroke-width="1.6" stroke-linecap="round"> <circle cx="20" cy="50" r="3" stroke="black" stroke-width="1" fill="black" /> <circle cx="38" cy="41" r="2" stroke="green" stroke-width="1"  /> <circle cx="2" cy="50" r="2" stroke="orange" stroke-width="1" fill=\'orange\' /> <circle cx="38" cy="59" r="2" stroke="orange" stroke-width="1"  /> <line x1="38" y1="41" x2="20" y2="50" stroke="green" stroke-width:2"   /><line x1="2" y1="50" x2="20" y2="50" stroke="orange" stroke-width:2"   /><line x1="38" y1="59" x2="20" y2="50" stroke="orange" stroke-width:2"   />  </g> </svg>', '<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="0" y="250"

In [36]:
generated_diags = graph_permutations(basis, loop_orders=[0,1], max_k=3)
len(generated_diags)

445

In [37]:
#diagram_set(generated_diags)

In [38]:
filtered = []
for _r in generated_diags:
    r = fgraph(_r)
    if r.is_1PI and r.betti_number() < 2:
        filtered.append(_r)
len(res), len(filtered)

#having_residual()
filtered = [f for f in filtered if f.residual_interaction == interactions[0]]
#hacing residual subset
print(len(filtered))

3


In [39]:
diagram_set(set(filtered))

<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="0" y="0" width="500" height="210">['<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="0" y="0" width="500" height="120">\n      <g fill="none" stroke="black" stroke-width="1.6" stroke-linecap="round"> <circle cx="70" cy="50" r="3" stroke="black" stroke-width="1" fill="black" /> <circle cx="20" cy="50" r="3" stroke="black" stroke-width="1" fill="black" /> <circle cx="52" cy="41" r="2" stroke="green" stroke-width="1" fill=\'green\' /> <circle cx="52" cy="59" r="2" stroke="orange" stroke-width="1" fill=\'orange\' /> <circle cx="88" cy="50" r="2" stroke="green" stroke-width="1"  /> <circle cx="38" cy="41" r="2" stroke="green" stroke-width="1"  /> <circle cx="38" cy="59" r="2" stroke="orange" stroke-width="1"  /> <circle cx="2" cy="50" r="2" stroke="green" stroke-width="1" fill=\'green\' /> <line x1="52" y1="41" x2="70" y2="50" stroke="green" stroke-width:2"  stroke-dasharray=\'2,2\' /><line x1="52" y1="41" x2="38" y2="41" stroke="green" stroke-width:2"  stroke-dasharray=\'2,2\' /><line x1="38" y1="41" x2="20" y2="50" stroke="green" stroke-width:2"  stroke-dasharray=\'2,2\' /><line x1="52" y1="59" x2="70" y2="50" stroke="orange" stroke-width:2"  stroke-dasharray=\'2,2\' /><line x1="52" y1="59" x2="38" y2="59" stroke="orange" stroke-width:2"  stroke-dasharray=\'2,2\' /><line x1="38" y1="59" x2="20" y2="50" stroke="orange" stroke-width:2"  stroke-dasharray=\'2,2\' /><line x1="88" y1="50" x2="70" y2="50" stroke="green" stroke-width:2"   /><line x1="2" y1="50" x2="20" y2="50" stroke="green" stroke-width:2"   />  </g> </svg>', '<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="0" y="50" width="500" height="120">\n      <g fill="none" stroke="black" stroke-width="1.6" stroke-linecap="round"> <circle cx="120" cy="50" r="3" stroke="black" stroke-width="1" fill="black" /> <circle cx="70" cy="50" r="3" stroke="black" stroke-width="1" fill="black" /> <circle cx="20" cy="50" r="3" stroke="black" stroke-width="1" fill="black" /> <circle cx="102" cy="41" r="2" stroke="green" stroke-width="1" fill=\'green\' /> <circle cx="102" cy="59" r="2" stroke="green" stroke-width="1" fill=\'green\' /> <circle cx="138" cy="50" r="2" stroke="green" stroke-width="1"  /> <circle cx="88" cy="50" r="2" stroke="green" stroke-width="1"  /> <circle cx="52" cy="50" r="2" stroke="orange" stroke-width="1" fill=\'orange\' /> <circle cx="38" cy="41" r="2" stroke="green" stroke-width="1"  /> <circle cx="38" cy="59" r="2" stroke="orange" stroke-width="1"  /> <circle cx="2" cy="50" r="2" stroke="green" stroke-width="1" fill=\'green\' /> <line x1="102" y1="41" x2="120" y2="50" stroke="green" stroke-width:2"  stroke-dasharray=\'2,2\' /><line x1="102" y1="41" x2="88" y2="50" stroke="green" stroke-width:2"  stroke-dasharray=\'2,2\' /><line x1="88" y1="50" x2="70" y2="50" stroke="green" stroke-width:2"  stroke-dasharray=\'2,2\' /><line x1="102" y1="59" x2="120" y2="50" stroke="green" stroke-width:2"  stroke-dasharray=\'2,2\' /><line x1="102" y1="59" x2="38" y2="41" stroke="green" stroke-width:2"  stroke-dasharray=\'2,2\' /><line x1="38" y1="41" x2="20" y2="50" stroke="green" stroke-width:2"  stroke-dasharray=\'2,2\' /><line x1="52" y1="50" x2="70" y2="50" stroke="orange" stroke-width:2"  stroke-dasharray=\'2,2\' /><line x1="52" y1="50" x2="38" y2="59" stroke="orange" stroke-width:2"  stroke-dasharray=\'2,2\' /><line x1="38" y1="59" x2="20" y2="50" stroke="orange" stroke-width:2"  stroke-dasharray=\'2,2\' /><line x1="138" y1="50" x2="120" y2="50" stroke="green" stroke-width:2"   /><line x1="2" y1="50" x2="20" y2="50" stroke="green" stroke-width:2"   />  </g> </svg>', '<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="0" y="100" width="500" height="120">\n      <g fill="none" stroke="black" stroke-width="1.6" stroke-linecap="round"> <ci

In [40]:
FG = filtered[0]

In [41]:
#takes tese examples which are sufficient to show some intereting loops
#define the propagators using the residual complement
filtered[1].residual_complement

array([[[0, 1],
        [0, 1]],

       [[1, 0],
        [1, 0]]])

In [46]:
from sympy import Symbol, symbols
from IPython.display import display, Math, Latex
from functools import reduce
from operator import add,mul
#THIS is all temporary just to get the ball rolling
#In reality, initially user would say some things about each species so that the form of the props can be determined
#Then, from the topology of the graph, we would find a basis in the momentum space that makes sense.
#I first need to determine some rules for this procedure.
class integral_repr(object):
    def __init__(self, propagators):
        
        display(Math(r'I = \int d^dk\ ' +str(reduce(mul, propagators, 1))))
    def __repr__(self):
        return ""
    
propagators = []
def propagator_from_species(s, p):
    D, k, i, om, T = symbols("D k i \omega, T")
    if s == -1:
        return T* propagator_from_species(0,1) * propagator_from_species(1,1)
    if s == 0: return (-1*i*om + D*k**2 + Symbol("m"+str(s)))**-p
    if s == 1: return (-1*i*om + Symbol("m"+str(s)))**-p

def internal_propagator_from_vertex_slice(s):
    if np.array_equal(s, np.array([[1,1],[0,0]])): return propagator_from_species(0,1)
    if np.array_equal(s, np.array([[0,0],[1,1]])): return propagator_from_species(1,1)
    if np.array_equal(s, np.array([[1,0],[0,1]])): return propagator_from_species(-1,1)
    
for v in filtered[1].residual_complement[1:-1]: #internal momenta at vertices
    propagators.append(internal_propagator_from_vertex_slice(v))
    
for i, s in enumerate(filtered[1].residual_complement[-1]):
    propagators.append(propagator_from_species(i, s[0]))

propagators

⎡         1                  1       ⎤
⎢────────────────────, ──────────────⎥
⎢   2                  -\omega⋅i + m₁⎥
⎣D⋅k  - \omega⋅i + m₀                ⎦

In [47]:
integral_repr(propagators)

<IPython.core.display.Math object>

# Distinct Loops

In [ ]:
#suppose we can group by something else afterwards e.g. residual equivalence - which contribute to renormalisation of say, propagators
#now we just need to determine the internal loop structure